# Initialization

Google Colab stuff

In [ ]:
# To access Google Drive files
from google.colab import drive
drive.mount('/content/drive')

# Install PyIrt from PyPi
!pip install pyirt

Import statements

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score
from pyirt import irt

# Import data

In [ ]:
df = pd.read_csv("/content/drive/My Drive/Recherche CS - etude de cas - Learning Analytics/data/public_data/train_data/train_task_1_2.csv")
df

,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue
0,16997,65967,12453206,0,4,2
1,16531,62121,15686710,1,1,1
2,15911,50013,13598796,0,3,1
3,1701,104909,10511925,0,4,3
4,22896,21748,941747,0,1,4
...,...,...,...,...,...,...
15867845,21324,117940,14754537,0,1,4
15867846,9191,62464,17477250,1,1,1
15867847,13091,71308,4786170,1,4,4
15867848,23065,38539,11284982,1,4,4


In [ ]:
df = df[["UserId", "QuestionId", "IsCorrect"]]

# IRT model

In [ ]:
# Transformation of the input as a list of tuples for IRT
scores = list(df.to_records(index=False))

In [ ]:
# IRT solver
item_param, user_param = irt(scores)

In [ ]:
def proba_irt(question_id, user_id, item_param=item_param, user_param=user_param):
    item = item_param[question_id]
    alpha, beta, c = item["alpha"], item["beta"], item["c"]
    theta = user_param[user_id]
    return c + (1-c)/(1+np.exp(-alpha*(theta + beta)))
def apply_proba(r):
    return proba_irt(r["QuestionId"], r["UserId"])

In [ ]:
df["Proba"] = df.apply(apply_proba, axis=1)

# Prediction

**Threshold of 0.5**

In [ ]:
THRESHOLD = 0.5
df["Pred"] = df.apply(lambda r: int(r["Proba"]>THRESHOLD), axis=1)

In [ ]:
print(classification_report(df["IsCorrect"], df["Pred"], digits=3))
print("AUC = {:.3f}".format(roc_auc_score(df["IsCorrect"], df["Proba"])))

              precision    recall  f1-score   support

           0      0.668     0.523     0.587   5665611
           1      0.764     0.855     0.807  10202239

    accuracy                          0.737  15867850
   macro avg      0.716     0.689     0.697  15867850
weighted avg      0.730     0.737     0.728  15867850

AUC = 0.798


**Threshold of 0.7**

In [ ]:
THRESHOLD = 0.7
df["Pred"] = df.apply(lambda r: int(r["Proba"]>THRESHOLD), axis=1)

In [ ]:
print(classification_report(df["IsCorrect"], df["Pred"], digits=3))
print("AUC = {:.3f}".format(roc_auc_score(df["IsCorrect"], df["Proba"])))

              precision    recall  f1-score   support

           0      0.554     0.814     0.659   5665611
           1      0.860     0.636     0.731  10202239

    accuracy                          0.699  15867850
   macro avg      0.707     0.725     0.695  15867850
weighted avg      0.751     0.699     0.705  15867850

AUC = 0.798
